<a href="https://colab.research.google.com/github/zqiaohe/CupIT2020/blob/master/%D0%A1%D0%BA%D0%BE%D0%BB%D1%8C%D0%B7%D1%8F%D1%89%D0%B0%D1%8F%D0%A1%D1%80%D0%B5%D0%B4%D0%BD%D1%8F%D1%8F.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_absolute_error

In [0]:
def dateparse (datetime):    
    return pd.Timestamp(datetime)
data = pd.read_csv('/content/drive/My Drive/CL_megafon_Cup IT_2020_final_data (1).csv', sep = ';', parse_dates = ['time_start', 'time_end'], date_parser=dateparse)

In [0]:
data['duration'] = data['time_end'] - data['time_start']
data['mins']= data['duration'].dt.seconds/60

In [0]:
sort_data = data.set_index('time_start').sort_index(ascending=False)

In [0]:
sort_data = sort_data.groupby(['place_latitude', 'place_longitude']).mins.rolling(window=2).mean()

In [10]:
sort_data

place_latitude  place_longitude  time_start         
27.002889       115.390675       2019-12-26 14:18:02           NaN
                                 2019-11-11 13:52:36    118.650000
                                 2019-08-08 23:30:36    207.283333
                                 2019-06-16 20:59:42    583.733333
                                 2019-06-15 08:56:25    573.041667
                                                           ...    
31.965556       113.599722       2018-03-29 00:38:20    106.266667
                                 2018-03-27 23:56:45    331.966667
31.977074       115.393131       2019-10-28 02:41:20           NaN
                                 2019-04-29 11:56:40    373.191667
                                 2019-02-15 10:33:24    247.750000
Name: mins, Length: 78223, dtype: float64

In [0]:
sort_data = sort_data.to_frame()
sort_data.reset_index(inplace=True)

In [12]:
sort_data

,place_latitude,place_longitude,time_start,mins
0,27.002889,115.390675,2019-12-26 14:18:02,NaN
1,27.002889,115.390675,2019-11-11 13:52:36,118.650000
2,27.002889,115.390675,2019-08-08 23:30:36,207.283333
3,27.002889,115.390675,2019-06-16 20:59:42,583.733333
4,27.002889,115.390675,2019-06-15 08:56:25,573.041667
...,...,...,...,...
78218,31.965556,113.599722,2018-03-29 00:38:20,106.266667
78219,31.965556,113.599722,2018-03-27 23:56:45,331.966667
78220,31.977074,115.393131,2019-10-28 02:41:20,NaN
78221,31.977074,115.393131,2019-04-29 11:56:40,373.191667


In [13]:
train = sort_data.merge(data, on = ['place_latitude', 'place_longitude', 'time_start'],suffixes=('2_mean', ''))
train

,place_latitude,place_longitude,time_start,mins2_mean,time_end,duration,mins
0,27.002889,115.390675,2019-12-26 14:18:02,NaN,2019-12-26 16:26:43,0 days 02:08:41,128.683333
1,27.002889,115.390675,2019-11-11 13:52:36,118.650000,2019-11-11 15:41:13,0 days 01:48:37,108.616667
2,27.002889,115.390675,2019-08-08 23:30:36,207.283333,2019-08-09 04:36:33,0 days 05:05:57,305.950000
3,27.002889,115.390675,2019-06-16 20:59:42,583.733333,2019-06-17 11:21:13,0 days 14:21:31,861.516667
4,27.002889,115.390675,2019-06-15 08:56:25,573.041667,2019-06-16 13:40:59,1 days 04:44:34,284.566667
...,...,...,...,...,...,...,...
78218,31.965556,113.599722,2018-03-29 00:38:20,106.266667,2018-03-29 01:09:00,0 days 00:30:40,30.666667
78219,31.965556,113.599722,2018-03-27 23:56:45,331.966667,2018-03-28 10:30:01,0 days 10:33:16,633.266667
78220,31.977074,115.393131,2019-10-28 02:41:20,NaN,2019-10-28 09:22:35,0 days 06:41:15,401.250000
78221,31.977074,115.393131,2019-04-29 11:56:40,373.191667,2019-04-29 17:41:48,0 days 05:45:08,345.133333


In [0]:
train = train.dropna()
train = train.drop_duplicates()

In [15]:
train

,place_latitude,place_longitude,time_start,mins2_mean,time_end,duration,mins
1,27.002889,115.390675,2019-11-11 13:52:36,118.650000,2019-11-11 15:41:13,0 days 01:48:37,108.616667
2,27.002889,115.390675,2019-08-08 23:30:36,207.283333,2019-08-09 04:36:33,0 days 05:05:57,305.950000
3,27.002889,115.390675,2019-06-16 20:59:42,583.733333,2019-06-17 11:21:13,0 days 14:21:31,861.516667
4,27.002889,115.390675,2019-06-15 08:56:25,573.041667,2019-06-16 13:40:59,1 days 04:44:34,284.566667
5,27.002889,115.390675,2019-04-06 05:18:01,202.266667,2019-04-06 07:17:59,0 days 01:59:58,119.966667
...,...,...,...,...,...,...,...
78217,31.965556,113.599722,2018-03-30 07:18:08,220.158333,2018-03-30 10:20:00,0 days 03:01:52,181.866667
78218,31.965556,113.599722,2018-03-29 00:38:20,106.266667,2018-03-29 01:09:00,0 days 00:30:40,30.666667
78219,31.965556,113.599722,2018-03-27 23:56:45,331.966667,2018-03-28 10:30:01,0 days 10:33:16,633.266667
78221,31.977074,115.393131,2019-04-29 11:56:40,373.191667,2019-04-29 17:41:48,0 days 05:45:08,345.133333


In [0]:
test_size = 0.2
test_index = int(len(train)*(1-test_size))
X_train = train.loc[:test_index].drop(['mins', 'duration', 'mins2_mean', 'time_end'], axis=1)
y_train = train.loc[:test_index]['mins2_mean']
X_test = train.loc[test_index:].drop(['mins', 'duration', 'mins2_mean', 'time_end'], axis=1)
y_test = train.loc[test_index:]['mins']

In [0]:
from sklearn.linear_model import LinearRegression

In [19]:
!pip install catboost
from catboost import CatBoostRegressor

     |████████████████████████████████| 64.4MB 45kB/s 


In [20]:
params = {'loss_function':'RMSE', 'learning_rate':1, 'depth':12}
model = CatBoostRegressor(iterations=1000,  **params, task_type='GPU')
model.fit(X_train, y_train)

0:	learn: 210.0679940	total: 302ms	remaining: 5m 1s
1:	learn: 206.7319454	total: 556ms	remaining: 4m 37s
2:	learn: 205.6399907	total: 806ms	remaining: 4m 27s
3:	learn: 204.9190213	total: 1.06s	remaining: 4m 23s
4:	learn: 203.9184105	total: 1.31s	remaining: 4m 21s
5:	learn: 203.0694048	total: 1.56s	remaining: 4m 19s
6:	learn: 201.5889582	total: 1.82s	remaining: 4m 17s
7:	learn: 200.6009813	total: 2.06s	remaining: 4m 16s
8:	learn: 199.8561582	total: 2.33s	remaining: 4m 16s
9:	learn: 199.6602537	total: 2.36s	remaining: 3m 53s
10:	learn: 198.8464864	total: 2.61s	remaining: 3m 54s
11:	learn: 198.3705094	total: 2.86s	remaining: 3m 55s
12:	learn: 198.1111022	total: 3.11s	remaining: 3m 56s
13:	learn: 197.7221457	total: 3.37s	remaining: 3m 57s
14:	learn: 197.5791864	total: 3.62s	remaining: 3m 57s
15:	learn: 197.0006188	total: 3.87s	remaining: 3m 57s
16:	learn: 196.3849535	total: 4.12s	remaining: 3m 58s
17:	learn: 195.9127930	total: 4.37s	remaining: 3m 58s
18:	learn: 195.5184416	total: 4.62s	rem

In [0]:
def mean_absolute_percentage_error(y_true, y_pred): 
    return 100 * (np.sum(np.abs((y_true - y_pred) / y_true))/len(y_true))

In [21]:
mean_absolute_percentage_error(y_test,  model.predict(X_test))

212.09338661840755